In [1]:
import os
import datetime
from datetime import timedelta
import quandl
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import plotnine as p9
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import zscore

import warnings
import functools
import wrds
import sys
# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
db = wrds.Connection()


WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
earnings_dates_for_tickers = pd.read_csv("earnings_dates_for_tickers.csv")
earnings_dates_for_tickers

,Unnamed: 0,index,Short_ticker,Name,Ticker,Date,Event Type,Adjusted Trading Date,trade_entry,trade_exit
0,0,396,A,Agilent Technologies Inc,A US,2018-02-14 16:05:00,ER,2018-02-15 00:00:00+00:00,2018-02-14 16:05:00,2018-02-15 16:05:00
1,1,1298,A,Agilent Technologies Inc,A US,2018-05-14 16:05:00,ER,2018-05-15 00:00:00+00:00,2018-05-14 16:05:00,2018-05-15 16:05:00
2,2,2023,A,Agilent Technologies Inc,A US,2018-08-14 16:05:00,ER,2018-08-15 00:00:00+00:00,2018-08-14 16:05:00,2018-08-15 16:05:00
3,3,2754,A,Agilent Technologies Inc,A US,2018-11-19 16:05:00,ER,2018-11-20 00:00:00+00:00,2018-11-19 16:05:00,2018-11-20 16:05:00
4,4,3297,A,Agilent Technologies Inc,A US,2019-02-20 16:05:00,ER,2019-02-21 00:00:00+00:00,2019-02-20 16:05:00,2019-02-21 16:05:00
...,...,...,...,...,...,...,...,...,...,...
19471,19471,15969,ZTS,Zoetis Inc,ZTS US,2023-02-14 07:00:00,ER,2023-02-13 00:00:00+00:00,2023-02-13 07:00:00,2023-02-14 07:00:00
19472,19472,16912,ZTS,Zoetis Inc,ZTS US,2023-05-04 07:00:00,ER,2023-05-03 00:00:00+00:00,2023-05-03 07:00:00,2023-05-04 07:00:00
19473,19473,17936,ZTS,Zoetis Inc,ZTS US,2023-08-08 07:00:00,ER,2023-08-07 00:00:00+00:00,2023-08-07 07:00:00,2023-08-08 07:00:00
19474,19474,18652,ZTS,Zoetis Inc,ZTS US,2023-11-02 07:00:00,ER,2023-11-01 00:00:00+00:00,2023-11-01 07:00:00,2023-11-02 07:00:00


In [4]:
csv_file_path = "secids.csv"

# Reading the CSV file into a NumPy array
secids = np.genfromtxt(csv_file_path, delimiter=',')
secids

array([101149., 205456., 210354., ..., 211262., 207854., 105959.])

In [5]:
options_dict = {} 
for year in range(2017, 2024):
    print(year)
    table_name = f"optionm.opprcd{year}"  # Generate table name dynamically
    query = f"""
    SELECT
        date, secid, symbol, cp_flag, expiry_indicator, volume, strike_price, exdate, open_interest, impl_volatility, best_bid, best_offer, delta, gamma, vega, theta
    FROM 
        {table_name} a 
    WHERE
        a.secid IN ({','.join(map(str, secids))}) AND
        a.volume > 0 AND
        a.exdate - a.date <= 7
    """
    options_dict[year] = db.raw_sql(query, date_cols=['date'])

2017
2018
2019
2020
2021
2022
2023


In [6]:
for year in range(2017, 2024):
    # Get the DataFrame for the current year
    options = options_dict[year]
    
    # Split the 'symbol' column and extract the first substring as 'ticker'
    options['ticker'] = options['symbol'].str.split(' ').str[0]
    
    # Set the multi-index using 'date' and 'ticker'
    options.set_index(['date', 'ticker'], inplace=True)
    
    # Update the options_dict with the modified DataFrame
    options_dict[year] = options

In [7]:
stock_prices = pd.read_parquet("stock_price_ED_20182023.parquet")
stock_prices

,date,permno,prc,bid,ask,ticker,comnam,permco
0,2018-02-07,87432,68.06,68.06000,68.07000,A,AGILENT TECHNOLOGIES INC,36364.0
1,2018-02-08,87432,65.05,65.03000,65.05000,A,AGILENT TECHNOLOGIES INC,36364.0
2,2018-02-09,87432,66.98,66.97000,66.98000,A,AGILENT TECHNOLOGIES INC,36364.0
3,2018-02-12,87432,68.43,68.40000,68.41000,A,AGILENT TECHNOLOGIES INC,36364.0
4,2018-02-13,87432,68.34,68.33000,68.34000,A,AGILENT TECHNOLOGIES INC,36364.0
...,...,...,...,...,...,...,...,...
147109,2023-11-03,13788,162.23,162.19000,162.23000,ZTS,ZOETIS INC,54327.0
147110,2023-11-06,13788,163.17,163.20000,163.27000,ZTS,ZOETIS INC,54327.0
147111,2023-11-07,13788,167.16,167.24001,167.28000,ZTS,ZOETIS INC,54327.0
147112,2023-11-08,13788,170.87,170.89000,170.92999,ZTS,ZOETIS INC,54327.0


In [8]:
earnings_dates_for_tickers['trade_entry'] = pd.to_datetime(earnings_dates_for_tickers['trade_entry'])

# Extract only the date part (YYYY-MM-DD)
earnings_dates_for_tickers['trade_entry'] = earnings_dates_for_tickers['trade_entry'].dt.date
earnings_dates_for_tickers['trade_entry']

0        2018-02-14
1        2018-05-14
2        2018-08-14
3        2018-11-19
4        2019-02-20
            ...    
19471    2023-02-13
19472    2023-05-03
19473    2023-08-07
19474    2023-11-01
19475    2024-02-12
Name: trade_entry, Length: 19476, dtype: object

In [9]:
earnings_dates_for_tickers['trade_entry'] = pd.to_datetime(earnings_dates_for_tickers['trade_entry'])
earnings_dates_for_tickers['trade_entry']


0       2018-02-14
1       2018-05-14
2       2018-08-14
3       2018-11-19
4       2019-02-20
           ...    
19471   2023-02-13
19472   2023-05-03
19473   2023-08-07
19474   2023-11-01
19475   2024-02-12
Name: trade_entry, Length: 19476, dtype: datetime64[ns]

In [10]:
earnings_dates_for_tickers['trade_exit'] = pd.to_datetime(earnings_dates_for_tickers['trade_exit'])

# Extract only the date part (YYYY-MM-DD)
earnings_dates_for_tickers['trade_exit'] = earnings_dates_for_tickers['trade_exit'].dt.date
earnings_dates_for_tickers['trade_exit'] = pd.to_datetime(earnings_dates_for_tickers['trade_exit'])
earnings_dates_for_tickers['trade_exit']



0       2018-02-15
1       2018-05-15
2       2018-08-15
3       2018-11-20
4       2019-02-21
           ...    
19471   2023-02-14
19472   2023-05-04
19473   2023-08-08
19474   2023-11-02
19475   2024-02-13
Name: trade_exit, Length: 19476, dtype: datetime64[ns]

In [11]:
stock_prices.set_index(['date','ticker'],inplace = True)
stock_prices

,,permno,prc,bid,ask,comnam,permco
date,ticker,,,,,,
2018-02-07,A,87432,68.06,68.06000,68.07000,AGILENT TECHNOLOGIES INC,36364.0
2018-02-08,A,87432,65.05,65.03000,65.05000,AGILENT TECHNOLOGIES INC,36364.0
2018-02-09,A,87432,66.98,66.97000,66.98000,AGILENT TECHNOLOGIES INC,36364.0
2018-02-12,A,87432,68.43,68.40000,68.41000,AGILENT TECHNOLOGIES INC,36364.0
2018-02-13,A,87432,68.34,68.33000,68.34000,AGILENT TECHNOLOGIES INC,36364.0
...,...,...,...,...,...,...,...
2023-11-03,ZTS,13788,162.23,162.19000,162.23000,ZOETIS INC,54327.0
2023-11-06,ZTS,13788,163.17,163.20000,163.27000,ZOETIS INC,54327.0
2023-11-07,ZTS,13788,167.16,167.24001,167.28000,ZOETIS INC,54327.0


In [12]:
def entry_stock_price(row):
    try:
        return stock_prices.loc[(row['trade_entry'], row['Short_ticker'])]['prc']
    except KeyError:
        return np.nan
earnings_dates_for_tickers['stock_price_entry'] = earnings_dates_for_tickers.apply(entry_stock_price,axis = 1)

In [13]:
def exit_stock_price(row):
    try:
        return stock_prices.loc[(row['trade_exit'], row['Short_ticker'])]['prc']
    except KeyError:
        return np.nan
earnings_dates_for_tickers['stock_price_exit'] = earnings_dates_for_tickers.apply(exit_stock_price,axis = 1)

In [14]:
earnings_dates_for_tickers['year_entry'] = earnings_dates_for_tickers['trade_entry'].dt.year

# Create 'year_exit' column containing the year of 'trade_exit'
earnings_dates_for_tickers['year_exit'] = earnings_dates_for_tickers['trade_exit'].dt.year
earnings_dates_for_tickers['year_entry']

0        2018
1        2018
2        2018
3        2018
4        2019
         ... 
19471    2023
19472    2023
19473    2023
19474    2023
19475    2024
Name: year_entry, Length: 19476, dtype: int32

In [26]:
approximation_error = abs(1000 * earnings_dates_for_tickers['stock_price_entry'] - 1000 * earnings_dates_for_tickers['stock_price_entry'].round())
approximation_error.mean()

245.41648013548803

In [37]:
options_dict[2017].columns

Index(['secid', 'symbol', 'cp_flag', 'expiry_indicator', 'volume',
       'strike_price', 'exdate', 'open_interest', 'impl_volatility',
       'best_bid', 'best_offer', 'delta', 'gamma', 'vega', 'theta'],
      dtype='object')

In [ ]:
def straddle_entry_mid(row):
    if row['year_entry'] < 2017 or row['year_entry'] > 2023 or row['year_exit'] < 2017 or row['year_exit'] > 2023:
        return np.nan
    try:
        options = options_dict[row['year_entry']].loc[(row['trade_entry'],row['Short_ticker'])]
    except:
        return np.nan
    options['strike_price'] *= 0.001

    # Calculate the absolute difference between 'strike_price' and row['stock_price_entry']
    options['abs_difference'] = abs(options['strike_price'] - row['stock_price_entry'])

    # Filter the DataFrame for rows with the minimum absolute difference
    min_abs_difference = options['abs_difference'].min()
    filtered_options = options[options['abs_difference'] == min_abs_difference]
    if len(filtered_options) ==2 and ('C' in filtered_options['cp_flag'].values) and ('P' in filtered_options['cp_flag'].values):
        return (filtered_options['best_bid'].mean()+filtered_options['best_offer'].mean())/2.0
    return np.nan
earnings_dates_for_tickers['straddle_entry_mid'] = earnings_dates_for_tickers.apply(straddle_entry_mid,axis = 1)

In [39]:
def straddle_entry(row):
    if row['year_entry'] < 2017 or row['year_entry'] > 2023 or row['year_exit'] < 2017 or row['year_exit'] > 2023:
        return np.nan*8
    try:
        options = options_dict[row['year_entry']].loc[(row['trade_entry'],row['Short_ticker'])]
    except:
        return np.nan*8

    # Calculate the absolute difference between 'strike_price' and row['stock_price_entry']
    options['abs_difference'] = abs(options['strike_price'] - row['stock_price_entry']*1000)

    # Filter the DataFrame for rows with the minimum absolute difference
    min_abs_difference = options['abs_difference'].min()
    filtered_options = options[options['abs_difference'] == min_abs_difference]
    if len(filtered_options) ==2 and ('C' in filtered_options['cp_flag'].values) and ('P' in filtered_options['cp_flag'].values):
        return filtered_options['impl_volatility'].mean(),filtered_options['best_bid'].sum(),filtered_options['best_offer'].sum(),filtered_options['gamma'].sum(),filtered_options['vega'].sum(),filtered_options['theta'].sum(),filtered_options['delta'].sum(),filtered_options['strike_price'].mean()
    return np.nan*8
earnings_dates_for_tickers[['straddle_entry_vol','straddle_entry_bid','straddle_entry_offer','straddle_entry_gamma','straddle_entry_vega','straddle_entry_theta','straddle_entry_delta','straddle_entry_strike']] = earnings_dates_for_tickers.apply(straddle_entry,axis = 1, result_type = 'expand')

In [16]:
earnings_dates_for_tickers['straddle_entry_mid'].count()

5566

In [41]:
def straddle_entry_skew_kurt(row):
    if row['year_entry'] < 2017 or row['year_entry'] > 2023 or row['year_exit'] < 2017 or row['year_exit'] > 2023:
        return np.nan*5
    try:
        options = options_dict[row['year_entry']].loc[(row['trade_entry'],row['Short_ticker'])]
    except:
        return np.nan*7
    IV_25c = np.nan
    IV_25p = np.nan
    IV_10c = np.nan
    IV_10p = np.nan
    IV_50c = np.nan
    # Calculate the absolute difference between 'strike_price' and row['stock_price_entry']
    options['delta25_c_diff'] = abs(options['delta'] - 0.25)
    options['delta25_p_diff'] = abs(options['delta'] + 0.25)
    options['delta10_c_diff'] = abs(options['delta'] - 0.1)
    options['delta10_p_diff'] = abs(options['delta'] + 0.1)
    options['delta50_c_diff'] = abs(options['delta'] - 0.5)
    min_delta25_c = options['delta25_c_diff'].min()
    min_delta25_p = options['delta25_p_diff'].min()
    min_delta10_c = options['delta10_c_diff'].min()
    min_delta10_p = options['delta10_p_diff'].min()
    min_delta50_c = options['delta50_c_diff'].min()
    if min_delta25_c <= 0.05:
        IV_25c = options[options['delta25_c_diff'] == min_delta25_c]['impl_volatility'].mean()
    if min_delta25_p <= 0.05:
        IV_25p = options[options['delta25_p_diff'] == min_delta25_p]['impl_volatility'].mean()
    if min_delta10_c <= 0.02:
        IV_10c = options[options['delta10_c_diff'] == min_delta10_c]['impl_volatility'].mean()
    if min_delta10_p <= 0.05:
        IV_10p = options[options['delta10_p_diff'] == min_delta10_p]['impl_volatility'].mean()
    if min_delta50_c <= 0.1:
        IV_50c = options[options['delta50_c_diff'] == min_delta50_c]['impl_volatility'].mean()
    return IV_25c,IV_25p,IV_10c,IV_10p,IV_50c
earnings_dates_for_tickers[['IV_25c','IV_25p','IV_10c','IV_10p','IV_50c']] = earnings_dates_for_tickers.apply(straddle_entry_skew_kurt,axis = 1, result_type = 'expand')

In [17]:
def straddle_exit_mid(row):
    if row['year_entry'] < 2017 or row['year_entry'] > 2023 or row['year_exit'] < 2017 or row['year_exit'] > 2023:
        return np.nan
    try:
        options = options_dict[row['year_exit']].loc[(row['trade_exit'],row['Short_ticker'])]
    except:
        return np.nan
    options['strike_price'] *= 0.001

    # Calculate the absolute difference between 'strike_price' and row['stock_price_entry']
    options['abs_difference'] = abs(options['strike_price'] - row['stock_price_entry'])

    # Filter the DataFrame for rows with the minimum absolute difference
    min_abs_difference = options['abs_difference'].min()
    filtered_options = options[options['abs_difference'] == min_abs_difference]
    if len(filtered_options) ==2 and ('C' in filtered_options['cp_flag'].values) and ('P' in filtered_options['cp_flag'].values):
        return (filtered_options['best_bid'].mean()+filtered_options['best_offer'].mean())/2.0
    return np.nan
earnings_dates_for_tickers['straddle_exit_mid'] = earnings_dates_for_tickers.apply(straddle_exit_mid,axis = 1)

In [18]:
earnings_dates_for_tickers['straddle_exit_mid'].count()

4693

In [19]:
earnings_dates_for_tickers['straddle_pnl'] = (earnings_dates_for_tickers['straddle_entry_mid'] - earnings_dates_for_tickers['straddle_exit_mid'])*1000.0/earnings_dates_for_tickers['straddle_entry_mid']

In [21]:
earnings_dates_for_tickers['straddle_pnl'].count()

4263

In [22]:
earnings_dates_for_tickers['straddle_pnl'].std()

720.5208311693931

In [30]:
earnings_dates_for_tickers.columns

Index(['Unnamed: 0', 'index', 'Short_ticker', 'Name', 'Ticker', 'Date',
       'Event Type', 'Adjusted Trading Date', 'trade_entry', 'trade_exit',
       'stock_price_entry', 'stock_price_exit', 'year_entry', 'year_exit',
       'straddle_entry_mid', 'straddle_exit_mid', 'straddle_pnl',
       'straddle_entry_bid', 'straddle_entry_offer', 'straddle_entry_gamma',
       'straddle_entry_vega', 'straddle_entry_theta', 'straddle_entry_delta',
       'straddle_entry_strike'],
      dtype='object')

In [36]:
earnings_dates_for_tickers[np.abs(earnings_dates_for_tickers['straddle_entry_strike']*0.001-earnings_dates_for_tickers['stock_price_entry'])/earnings_dates_for_tickers['stock_price_entry'] < 0.05]['straddle_pnl'].count()

1423

In [42]:
earnings_dates_for_tickers['skew_25'] = 4.4478*(earnings_dates_for_tickers['IV_25c']-earnings_dates_for_tickers['IV_25p'])/earnings_dates_for_tickers['IV_50c']

In [47]:
earnings_dates_for_tickers['kurt_25'] = 52.7546*(0.5*(earnings_dates_for_tickers['IV_25c']+earnings_dates_for_tickers['IV_25p'])/earnings_dates_for_tickers['IV_50c'] - 1.0)
earnings_dates_for_tickers['kurt_25'].count()

5594

In [43]:
earnings_dates_for_tickers['skew_25'].count()

5594

In [44]:
(earnings_dates_for_tickers[['straddle_pnl', 'skew_25']].notna().all(axis=1)).sum()

3613

In [48]:
(earnings_dates_for_tickers[['straddle_pnl', 'kurt_25']].notna().all(axis=1)).sum()

3613

In [45]:
num_quantiles = 10

# Create deciles based on 'skew_25'
earnings_dates_for_tickers['skew_25_decile'] = pd.qcut(earnings_dates_for_tickers['skew_25'], num_quantiles)

# Calculate the mean 'straddle_pnl' for each decile
mean_straddle_pnl_by_decile = earnings_dates_for_tickers.groupby('skew_25_decile')['straddle_pnl'].mean()

print(mean_straddle_pnl_by_decile)

skew_25_decile
(-20.905, -0.762]    45.359169
(-0.762, -0.586]     22.012424
(-0.586, -0.468]    -20.934392
(-0.468, -0.368]     18.731875
(-0.368, -0.272]     25.583620
(-0.272, -0.171]    -32.543577
(-0.171, -0.0677]    73.838410
(-0.0677, 0.0608]    75.767279
(0.0608, 0.263]     -20.306298
(0.263, 11.769]     -52.400244
Name: straddle_pnl, dtype: float64


In [46]:
earnings_dates_for_tickers[earnings_dates_for_tickers['skew_25']<0.0608]['straddle_pnl'].mean()

25.82727723211073

In [49]:
num_quantiles = 10

# Create deciles based on 'skew_25'
earnings_dates_for_tickers['kurt_25_decile'] = pd.qcut(earnings_dates_for_tickers['kurt_25'], num_quantiles)

# Calculate the mean 'straddle_pnl' for each decile
mean_straddle_pnl_by_kurt_decile = earnings_dates_for_tickers.groupby('kurt_25_decile')['straddle_pnl'].mean()

print(mean_straddle_pnl_by_kurt_decile)

kurt_25_decile
(-27.842000000000002, -3.017]    28.220626
(-3.017, -1.869]                -43.454418
(-1.869, -1.159]                 14.486252
(-1.159, -0.614]                -10.801209
(-0.614, -0.0246]                16.087484
(-0.0246, 0.593]                  8.004303
(0.593, 1.279]                   -4.798380
(1.279, 2.184]                   32.077277
(2.184, 3.869]                   40.361388
(3.869, 452.588]                 67.473452
Name: straddle_pnl, dtype: float64


In [52]:
num_quantiles = 10

# Create deciles based on 'skew_25'
earnings_dates_for_tickers['straddle_vol_decile'] = pd.qcut(earnings_dates_for_tickers['straddle_entry_vol'], num_quantiles)

# Calculate the mean 'straddle_pnl' for each decile
mean_straddle_pnl_by_vol_decile = earnings_dates_for_tickers.groupby('straddle_vol_decile')['straddle_pnl'].mean()

print(mean_straddle_pnl_by_vol_decile)

straddle_vol_decile
(0.206, 0.501]   -69.254183
(0.501, 0.621]     0.756881
(0.621, 0.73]     23.058441
(0.73, 0.849]     17.395843
(0.849, 0.952]    43.000185
(0.952, 1.104]    45.939242
(1.104, 1.293]   -27.507660
(1.293, 1.566]   -41.562691
(1.566, 1.957]     2.208024
(1.957, 2.982]    67.354911
Name: straddle_pnl, dtype: float64


In [50]:
earnings_dates_for_tickers[earnings_dates_for_tickers['kurt_25']>=1.279]['straddle_pnl'].mean()

45.867461955398504

In [51]:
earnings_dates_for_tickers.to_csv("results.csv")